In [2]:
import os
import pandas as pd
from torch.utils.data import Dataset
from sklearn.preprocessing import StandardScaler
# from utils.timefeatures import time_features
import warnings

warnings.filterwarnings('ignore')


In [ ]:
def __init__(self, root_path, flag='train', size=None,
                features='S', data_path='ETTh1.csv',
                target='OT', scale=True, timeenc=0, freq='h'):
    # size [seq_len, label_len, pred_len]
    # info
    if size == None:
        self.seq_len = 24 * 4 * 4
        self.label_len = 24 * 4
        self.pred_len = 24 * 4
    else:
        self.seq_len = size[0]
        self.label_len = size[1]
        self.pred_len = size[2]
    # init
    assert flag in ['train', 'test', 'val']
    type_map = {'train': 0, 'val': 1, 'test': 2}
    self.set_type = type_map[flag]

    self.features = features
    self.target = target
    self.scale = scale
    self.timeenc = timeenc
    self.freq = freq

    self.root_path = root_path
    self.data_path = data_path
    self.__read_data__()

In [4]:
scaler = StandardScaler()
df_raw = pd.read_csv('/home/../multiTS/NFT/data/informer_datasets/weather/weather.csv')
''' df_raw.columns: ['date', ...(other features), target feature] '''
cols = list(df_raw.columns)

In [6]:
cols

['date',
 'p (mbar)',
 'T (degC)',
 'Tpot (K)',
 'Tdew (degC)',
 'rh (%)',
 'VPmax (mbar)',
 'VPact (mbar)',
 'VPdef (mbar)',
 'sh (g/kg)',
 'H2OC (mmol/mol)',
 'rho (g/m**3)',
 'wv (m/s)',
 'max. wv (m/s)',
 'wd (deg)',
 'rain (mm)',
 'raining (s)',
 'SWDR (W/m�)',
 'PAR (�mol/m�/s)',
 'max. PAR (�mol/m�/s)',
 'Tlog (degC)',
 'OT']

In [3]:
def __read_data__():
    scaler = StandardScaler()
    df_raw = pd.read_csv('/home/../multiTS/NFT/data/informer_datasets/weather/weather.csv')
    ''' df_raw.columns: ['date', ...(other features), target feature] '''
    cols = list(df_raw.columns)
    cols.remove(self.target)
    cols.remove('date')
    df_raw = df_raw[['date'] + cols + [self.target]]
    # print(cols)
    num_train = int(len(df_raw) * 0.7)
    num_test = int(len(df_raw) * 0.2)
    num_vali = len(df_raw) - num_train - num_test
    border1s = [0, num_train - self.seq_len, len(df_raw) - num_test - self.seq_len]
    border2s = [num_train, num_train + num_vali, len(df_raw)]
    border1 = border1s[self.set_type]
    border2 = border2s[self.set_type]

    if self.features == 'M' or self.features == 'MS':
        cols_data = df_raw.columns[1:]
        df_data = df_raw[cols_data]
    elif self.features == 'S':
        df_data = df_raw[[self.target]]

    if self.scale:
        train_data = df_data[border1s[0]:border2s[0]]
        self.scaler.fit(train_data.values)
        data = self.scaler.transform(df_data.values)
    else:
        data = df_data.values

    df_stamp = df_raw[['date']][border1:border2]
    df_stamp['date'] = pd.to_datetime(df_stamp.date)
    if self.timeenc == 0:
        df_stamp['month'] = df_stamp.date.apply(lambda row: row.month, 1)
        df_stamp['day'] = df_stamp.date.apply(lambda row: row.day, 1)
        df_stamp['weekday'] = df_stamp.date.apply(lambda row: row.weekday(), 1)
        df_stamp['hour'] = df_stamp.date.apply(lambda row: row.hour, 1)
        data_stamp = df_stamp.drop(['date'], 1).values
    elif self.timeenc == 1:
        data_stamp = time_features(pd.to_datetime(df_stamp['date'].values), freq=self.freq)
        data_stamp = data_stamp.transpose(1, 0)

    self.data_x = data[border1:border2]
    self.data_y = data[border1:border2]
    self.data_stamp = data_stamp

    